# Limitaciones del Modelo de Regímenes

**Notebook 2 de 3** | Análisis Crítico del Enfoque Markoviano

En este notebook examinamos las limitaciones teóricas y empíricas del modelo:
1. Propiedades de la matriz de transición (supuestos Markovianos)
2. Evidencia de violación: no hay memoria de duración
3. Volatility clustering (ACF de retornos vs |retornos|)
4. Fat tails: comparación con Normal y t-Student
5. Drift temporal: estabilidad de centroides IS vs OOS

---

## 0. Setup y Carga de Datos

In [ ]:
# Para Google Colab: descomentar y ejecutar
# !pip install -q yfinance statsmodels networkx
# !git clone https://github.com/tu-usuario/blog_regimenes.git
# %cd blog_regimenes/notebooks

In [ ]:
import sys
sys.path.append("..")

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from statsmodels.graphics.tsaplots import plot_acf

from config import *
from src.data import download_spy_data, train_test_split
from src.features import compute_oliva_features
from src.regimes import (
    standardize_features, fit_kmeans, 
    compute_transition_matrix, compute_centroid_distances
)
from src.plotting import (
    setup_plotting_style, save_figure,
    plot_acf_comparison, plot_distribution_comparison
)

setup_plotting_style()

SAVE_FIGURES = True
FIGURES_PATH = "../figures/post2/"

print("Configuración cargada")

In [ ]:
# Re-entrenar modelo (mismo código que notebook 1 para independencia)
# TODO: Descargar datos y entrenar modelo
# df = download_spy_data()
# df_train, df_test = train_test_split(df)
# features_train = compute_oliva_features(df_train)
# features_scaled, scaler = standardize_features(features_train)
# model, labels = fit_kmeans(features_scaled, k=K_REGIMES)

---
## 1. Propiedades de la Matriz de Transición

### Supuestos del modelo Markoviano

La matriz de transición asume:

1. **Propiedad de Markov (primer orden):** $P(X_{t+1} | X_t, X_{t-1}, ...) = P(X_{t+1} | X_t)$
   - El estado futuro solo depende del estado actual
   
2. **Estacionariedad:** Las probabilidades de transición no cambian en el tiempo

3. **Sin memoria de duración:** El tiempo que llevamos en un estado no afecta la probabilidad de transición
   - Si llevamos 5 días en Bull, la prob. de salir es la misma que si llevamos 50 días

### 1.1 Verificación empírica: ¿Hay memoria de duración?

Si el modelo Markoviano es correcto, la duración de un régimen debería seguir una distribución geométrica.

In [ ]:
def compute_regime_durations(labels: np.ndarray) -> dict:
    """
    Calcula la duración de cada visita a cada régimen.
    
    Returns
    -------
    dict : {regime: [durations]}
    """
    durations = {i: [] for i in range(K_REGIMES)}
    
    current_regime = labels[0]
    current_duration = 1
    
    for i in range(1, len(labels)):
        if labels[i] == current_regime:
            current_duration += 1
        else:
            durations[current_regime].append(current_duration)
            current_regime = labels[i]
            current_duration = 1
    
    # Añadir última duración
    durations[current_regime].append(current_duration)
    
    return durations

# TODO: Calcular duraciones
# durations = compute_regime_durations(labels)

In [ ]:
# TODO: Comparar distribución empírica vs geométrica teórica
# Para cada régimen:
# - p_stay = prob. diagonal de la matriz de transición
# - Duración teórica ~ Geom(1 - p_stay), con media = 1/(1-p_stay)

In [ ]:
def plot_duration_analysis(durations: dict, trans_matrix: np.ndarray, save_path=None):
    """
    Compara distribución empírica de duraciones vs geométrica teórica.
    """
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    axes = axes.flatten()
    
    for regime in range(K_REGIMES):
        ax = axes[regime]
        dur = durations[regime]
        
        if len(dur) > 0:
            # Histograma empírico
            max_dur = max(dur)
            bins = range(1, min(max_dur + 2, 50))
            ax.hist(dur, bins=bins, density=True, alpha=0.7, 
                   label='Empírica', color=REGIME_COLORS[regime])
            
            # Geométrica teórica
            p_stay = trans_matrix[regime, regime]
            p_leave = 1 - p_stay
            x = np.arange(1, 50)
            geom_pmf = stats.geom.pmf(x, p_leave)
            ax.plot(x, geom_pmf, 'k--', linewidth=2, label=f'Geom(p={p_leave:.3f})')
            
            ax.set_title(f"{REGIME_NAMES[regime]}\nMedia emp: {np.mean(dur):.1f}, teórica: {1/p_leave:.1f}")
            ax.set_xlabel('Duración (días)')
            ax.set_ylabel('Densidad')
            ax.legend()
            ax.set_xlim(0, 40)
    
    plt.suptitle('Distribución de Duraciones: Empírica vs Geométrica', fontsize=14)
    plt.tight_layout()
    
    if save_path:
        save_figure(fig, save_path)
    
    return fig

# TODO: Ejecutar análisis
# trans_matrix = compute_transition_matrix(labels, k=K_REGIMES)
# save_path = f"{FIGURES_PATH}duraciones_vs_geometrica.png" if SAVE_FIGURES else None
# plot_duration_analysis(durations, trans_matrix, save_path)

### Interpretación

TODO: Discutir si la distribución empírica se ajusta a la geométrica:
- Si hay colas más pesadas → los regímenes son más persistentes de lo que predice Markov
- Implicaciones para el modelo

---
## 2. Volatility Clustering

Una propiedad bien documentada de los retornos financieros: la volatilidad tiende a agruparse en el tiempo.

In [ ]:
# TODO: Comparar ACF de retornos vs ACF de |retornos|
# save_path = f"{FIGURES_PATH}acf_comparacion.png" if SAVE_FIGURES else None
# plot_acf_comparison(df_train['returns'], lags=50, save_path=save_path)

### Interpretación

TODO: Explicar el patrón observado:
- ACF de retornos: cercana a cero (no hay autocorrelación significativa)
- ACF de |retornos|: decae lentamente (volatility clustering)
- ¿Qué implica esto para nuestro modelo de regímenes?

---
## 3. Distribución de Retornos: Fat Tails

In [ ]:
# TODO: Comparar con Normal y t-Student
# save_path = f"{FIGURES_PATH}distribucion_retornos.png" if SAVE_FIGURES else None
# plot_distribution_comparison(df_train['returns'], save_path=save_path)

In [ ]:
# TODO: Test de normalidad
# from scipy.stats import jarque_bera, shapiro
# 
# jb_stat, jb_pval = jarque_bera(df_train['returns'].dropna())
# print(f"Jarque-Bera: stat={jb_stat:.2f}, p-value={jb_pval:.2e}")

In [ ]:
# TODO: Comparar momentos empíricos vs distribuciones ajustadas
# returns = df_train['returns'].dropna()
# 
# moments = {
#     'Empírica': [returns.mean(), returns.std(), returns.skew(), returns.kurtosis()],
#     'Normal': [returns.mean(), returns.std(), 0, 0],  # Skew=0, Kurt=0 para normal
#     't-Student': [...]  # Calcular después del ajuste
# }
# 
# pd.DataFrame(moments, index=['Media', 'Std', 'Skewness', 'Kurtosis']).round(4)

### Interpretación

TODO: Discutir:
- Exceso de kurtosis (fat tails)
- Grados de libertad del ajuste t-Student
- Implicaciones para gestión de riesgo

---
## 4. QQ-Plots Detallados

In [ ]:
def plot_qq_comparison(returns, save_path=None):
    """
    QQ-plots comparando retornos con Normal y t-Student.
    """
    fig, axes = plt.subplots(1, 2, figsize=(12, 5))
    
    returns_clean = returns.dropna()
    
    # QQ vs Normal
    stats.probplot(returns_clean, dist="norm", plot=axes[0])
    axes[0].set_title("QQ-Plot vs Normal")
    axes[0].get_lines()[0].set_markerfacecolor('blue')
    axes[0].get_lines()[0].set_alpha(0.5)
    
    # QQ vs t-Student (ajustada)
    df_t, loc_t, scale_t = stats.t.fit(returns_clean)
    stats.probplot(returns_clean, dist=stats.t, sparams=(df_t,), plot=axes[1])
    axes[1].set_title(f"QQ-Plot vs t-Student (df={df_t:.1f})")
    axes[1].get_lines()[0].set_markerfacecolor('green')
    axes[1].get_lines()[0].set_alpha(0.5)
    
    plt.tight_layout()
    
    if save_path:
        save_figure(fig, save_path)
    
    return fig

# TODO: Ejecutar
# save_path = f"{FIGURES_PATH}qq_plots.png" if SAVE_FIGURES else None
# plot_qq_comparison(df_train['returns'], save_path)

---
## 5. Drift: Estabilidad de Centroides IS vs OOS

In [ ]:
# TODO: Aplicar modelo a datos OOS
# features_test = compute_oliva_features(df_test)
# features_test_scaled, _ = standardize_features(features_test, scaler=scaler)
# labels_test = model.predict(features_test_scaled)

In [ ]:
# TODO: Calcular distancias a centroides
# dist_train = compute_centroid_distances(features_scaled, model)
# dist_test = compute_centroid_distances(features_test_scaled, model)

In [ ]:
def plot_centroid_distances_comparison(dist_train, dist_test, labels_train, labels_test, save_path=None):
    """
    Compara distribución de distancias a centroides IS vs OOS.
    """
    fig, axes = plt.subplots(2, 2, figsize=(12, 10))
    axes = axes.flatten()
    
    for regime in range(K_REGIMES):
        ax = axes[regime]
        col = f'dist_centroid_{regime}'
        
        # Solo puntos asignados a este régimen
        train_dist = dist_train.loc[labels_train == regime, col]
        test_dist = dist_test.loc[labels_test == regime, col]
        
        ax.hist(train_dist, bins=30, density=True, alpha=0.6, label='In-Sample', color='blue')
        ax.hist(test_dist, bins=30, density=True, alpha=0.6, label='Out-of-Sample', color='red')
        
        ax.set_title(f"{REGIME_NAMES[regime]}")
        ax.set_xlabel('Distancia al centroide')
        ax.set_ylabel('Densidad')
        ax.legend()
        
        # Añadir estadísticas
        ax.axvline(train_dist.mean(), color='blue', linestyle='--', alpha=0.8)
        ax.axvline(test_dist.mean(), color='red', linestyle='--', alpha=0.8)
    
    plt.suptitle('Distancia a Centroides: IS vs OOS (Drift Analysis)', fontsize=14)
    plt.tight_layout()
    
    if save_path:
        save_figure(fig, save_path)
    
    return fig

# TODO: Ejecutar
# save_path = f"{FIGURES_PATH}drift_centroides.png" if SAVE_FIGURES else None
# plot_centroid_distances_comparison(dist_train, dist_test, labels, labels_test, save_path)

In [ ]:
# TODO: Test estadístico de drift (Kolmogorov-Smirnov)
# from scipy.stats import ks_2samp
# 
# drift_results = []
# for regime in range(K_REGIMES):
#     col = f'dist_centroid_{regime}'
#     train_dist = dist_train.loc[labels == regime, col]
#     test_dist = dist_test.loc[labels_test == regime, col]
#     
#     ks_stat, ks_pval = ks_2samp(train_dist, test_dist)
#     drift_results.append({
#         'Régimen': REGIME_NAMES[regime],
#         'Media IS': train_dist.mean(),
#         'Media OOS': test_dist.mean(),
#         'KS Stat': ks_stat,
#         'KS p-value': ks_pval
#     })
# 
# pd.DataFrame(drift_results)

### Interpretación

TODO: Discutir:
- ¿Hay drift significativo entre IS y OOS?
- ¿Qué regímenes son más estables?
- Implicaciones para uso en producción

---
## 6. Resumen de Limitaciones

TODO: Tabla resumen de limitaciones identificadas:

| Limitación | Evidencia | Impacto |
|------------|-----------|--------|
| Sin memoria de duración | ... | ... |
| Volatility clustering | ... | ... |
| Fat tails | ... | ... |
| Drift temporal | ... | ... |

---
## 7. Conclusiones del Notebook 2

TODO: Resumir:
- Principales violaciones de los supuestos
- Implicaciones prácticas
- Alternativas a explorar (Jump Models, HSMM, etc.)

**Siguiente notebook:** Backtest y extensiones del modelo.